# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Свои функции
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender

In [2]:
data = pd.read_csv('../data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features = pd.read_csv('../data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(3)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."
2,5,"[913077, 1118028, 1386668]"


In [5]:
MainRec = MainRecommender(data_train, item_features)

/home/mayo889/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [6]:
%%time

result['similar_items_recs'] = result['user_id'].apply(lambda x: MainRec.get_similar_items_recommendation(x, N=5))

CPU times: user 38.8 s, sys: 1.61 s, total: 40.4 s
Wall time: 17.7 s


In [7]:
%%time

result['similar_users_recs'] = result['user_id'].apply(lambda x: MainRec.get_similar_users_recommendation(x, N=5))

CPU times: user 1min 14s, sys: 2.68 s, total: 1min 17s
Wall time: 38.7 s


In [8]:
%%time

result['user_recs'] = result['user_id'].apply(lambda x: MainRec.user_recommendation(x, N=5))

CPU times: user 48.7 s, sys: 1.77 s, total: 50.5 s
Wall time: 25.3 s


In [9]:
result.head()

,user_id,actual,similar_items_recs,similar_users_recs,user_recs
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[1082185, 866211, 981760, 1082185, 5582712]","[951590, 1082185, 1029743, 1122358, 833598]","[874617, 999391, 961021, 885290, 7025106]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1082185, 981760, 1076875, 1037894, 1092026]","[1082185, 986912, 893501, 981760, 995242]","[8090537, 1110244, 8090521, 883404, 5568378]"
2,5,"[913077, 1118028, 1386668]","[1082185, 981760, 12262778, 1070702, 833025]","[1029743, 1082185, 839849, 995242, 1088835]","[849202, 1075368, 860248, 1034176, 959737]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108...","[1082185, 904360, 866211, 981760, 1004906]","[883404, 1106523, 1133018, 1082185, 7167218]","[1082185, 857006, 999250, 878996, 965267]"
4,7,"[929248, 948622, 1013572, 1022003, 1049892, 10...","[1082185, 866211, 981760, 1015247, 1050229]","[1133018, 849843, 883404, 1043751, 1082185]","[853643, 951703, 938187, 839419, 1130111]"


In [10]:
# Результат, если отфильтровать только популярные товары

comparing = {}

for name_col in result.columns[1:]:
    comparing[name_col] = round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k=5),
                                             axis=1).mean(), 4)

comparing = pd.DataFrame.from_dict(comparing, orient='index', columns=['precision@5'])
comparing.sort_values(by='precision@5', ascending=False, inplace=True)
comparing

,precision@5
actual,1.0000
similar_items_recs,0.1527
user_recs,0.1225
similar_users_recs,0.1187


In [10]:
# Результат, если отфильтровать дешевые, дорогие и популярные товары

comparing = {}

for name_col in result.columns[1:]:
    comparing[name_col] = round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k=5),
                                             axis=1).mean(), 4)

comparing = pd.DataFrame.from_dict(comparing, orient='index', columns=['precision@5'])
comparing.sort_values(by='precision@5', ascending=False, inplace=True)
comparing

,precision@5
actual,1.0000
user_recs,0.0947
similar_items_recs,0.0603
similar_users_recs,0.0465


In [10]:
# Результат, если отфильтровать дешевые, дорогие, популярные товары и неинтересные категории

comparing = {}

for name_col in result.columns[1:]:
    comparing[name_col] = round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k=5),
                                             axis=1).mean(), 4)

comparing = pd.DataFrame.from_dict(comparing, orient='index', columns=['precision@5'])
comparing.sort_values(by='precision@5', ascending=False, inplace=True)
comparing

,precision@5
actual,1.0000
user_recs,0.0913
similar_items_recs,0.0555
similar_users_recs,0.0492


In [10]:
# Результат, если отфильтровать популярные товары и неинтересные категории

comparing = {}

for name_col in result.columns[1:]:
    comparing[name_col] = round(result.apply(lambda row: precision_at_k(row[name_col], row['actual'], k=5),
                                             axis=1).mean(), 4)

comparing = pd.DataFrame.from_dict(comparing, orient='index', columns=['precision@5'])
comparing.sort_values(by='precision@5', ascending=False, inplace=True)
comparing

,precision@5
actual,1.0000
similar_items_recs,0.1575
user_recs,0.1208
similar_users_recs,0.1105
